# AGLAE PIGE data treatment

In [ ]:
%pip install ipywidgets xlrd

In [ ]:
import bae
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import io
import pandas as pd
from pandas.api.typing import DataFrameGroupBy
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
uploader_report = widgets.FileUpload()
display(uploader_report)

In [ ]:
uploader = widgets.FileUpload()
display(uploader)

In [ ]:
print(uploader.value)
print(uploader_report.value)
pige_table = pd.read_csv(io.BytesIO(uploader.value[0].content), sep="\t")
report = pd.read_excel(io.BytesIO(uploader_report.value[0].content))

# On garde les tableaux avec uniquement les standards

In [ ]:
stds = pd.DataFrame(
    {
         "Nom std": ["BrillA", "BrillB", "BrillC", "BrillD", "BG4", "BG3"],
         "Conc theorique": [143900, 172600, 12000, 10700, 50000, 75000],
    }
)

nb_stds = len(stds)

report_stds = report[report["Ref Objet"].isin(stds["Nom std"])]
pige_stds = pige_table[pige_table["Nom Fichier"].str.contains("|".join(stds["Nom std"]), regex=True)]

# conversion des colonnes de durées
#report_stds["Time"] = pd.to_timedelta(report_stds["Time"])
#report_stds["Analyse time"] = pd.to_timedelta(report_stds["Analyse time"])

# on vérifie que les tableaux report et pige_stds ont bien un nombre de lignes multiple du nombre de standards (6 en général)

if len(report_stds)%nb_stds == 0:
    print(f"youpi: la longueur du tableau report est bien un multiple de {nb_stds}")
else:
    print("fatal error: problème de longueur du tableau report")

if len(pige_stds)%nb_stds == 0:
    print(f"youpi: la longueur du tableau pige_stds est bien un multiple de {nb_stds}")
else:
    print("fatal error: problème de longueur du tableau pige_stds")
    

# Calcul Aire Gaussienne / Dose
On calcule l'aire sur dose pour tracer la courbe de calibration PIGE

In [ ]:
ad = pige_stds["Aire Gaussien"]/report_stds["Dose"]
nb_analyses = int(len(report_stds)/nb_stds)

ad_stds = pd.DataFrame(
    {
        "Nom std": report_stds["Ref Objet"], 
        "A/D": ad, 
    }
)

# On crée une liste de dataframe pour chaque analyse de standards de la journée, avec et sans Pb
ad_stds_sansPb = []
ad_stds_avecPb = []

# On fait une boucle for pour parcourir le tableau ad par bloc du nombre de standards et extraire ceux avec du Pb et ceux sans
for i in range(0, len(ad_stds), nb_stds):
    ad_stds_i = ad_stds[i:i+nb_stds]
    ad_stds_sansPb.append(ad_stds_i[ad_stds_i["Nom std"].isin(["BrillA","BrillB","BrillD"])])
    ad_stds_avecPb.append(ad_stds_i[ad_stds_i["Nom std"].isin(["BrillC","BG4","BG3"])])

conc_stds_sansPb = stds[stds["Nom std"].isin(["BrillA","BrillB","BrillD"])]
conc_stds_avecPb = stds[stds["Nom std"].isin(["BrillC","BG4","BG3"])]

print(ad_stds)
print(ad_stds_sansPb)
print(conc_stds_sansPb)
print(conc_stds_avecPb)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,4))

# Fonction qui va afficher les courbes de calibration et son fit linéaire par méthode des moindres carrés ordinaires
def calib(titre, ad, conc, plot_idx):
    axes[plot_idx].set_title(titre)
    axes[plot_idx].set_xlabel("Aire / Dose (u.a.)")
    axes[plot_idx].set_ylabel("Concentration théorique (ppm)")
    axes[plot_idx].grid(True)
    color = ["blue", "red"]
    
    # Boucle for pour afficher toutes les droites de calibration sans plomb
    for i, e in enumerate(ad):
        x = e["A/D"]
        y = conc["Conc theorique"]    
                  
        axes[plot_idx].plot(e["A/D"],conc["Conc theorique"], '.', label=f"Analyse {i}", markersize=12, color=color[i]) 
        
        # Ajustement d'une droite passant par l'origine (y = ax)
        coef, _, _, _ = np.linalg.lstsq(np.array(x).reshape(-1, 1), np.array(y), rcond=None)
    
        
        # Prédictions de y pour cette droite ajustée
        y_pred = coef[0] * np.array(x)
    
        # Calcul du coefficient de détermination R²
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r_squared = 1 - (ss_res / ss_tot)
    
        # Affichage de la droite ajustée
        axes[plot_idx].plot(x, y_pred, label=f"Droite {i}: y={coef[0]:.2f}x\nR²={r_squared:.2f}", linestyle='--')
        
    axes[plot_idx].legend(loc='best', fontsize=10)

calib("Courbe de calibration Na PIGE sans Plomb", ad_stds_sansPb, conc_stds_sansPb, 0)
calib("Courbe de calibration Na PIGE avec Plomb", ad_stds_avecPb, conc_stds_avecPb, 1)

# Extraction de la colonne "Time" et "Analyse time" de report
for i in range(0, len(report_stds), nb_stds):
    time_stds_i = pd.to_timedelta(report_stds["Time"][i:i+nb_stds])
    print(time_stds_i)
    print(time_stds_i.iloc[0], time_stds_i.iloc[-1], time_stds_i.iloc[-1] - time_stds_i.iloc[0])
    #print(i+nb_stds-1, time_stds_i.iloc[i+nb_stds-1])
    #print(time_stds_i[i+nb_stds-1] - time_stds_i[i])
